https://kimjj81.github.io/2025/06/13/RAG-%EC%A7%88%EC%9D%98-%ED%99%95%EC%9E%A5%EA%B3%BC-%EC%9E%AC%EC%88%9C%EC%9C%84%ED%99%94/ 의 예제

먼저 아래 코드를 실행하여 필요한 라이브러리를 설치

In [1]:
!pip install sentence-transformers torch transformers


아래 예시는 sentence-transformers 임베딩 모델을 사용해서 질의와 유사한 키워드를 찾아 확장하는 예

In [2]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

query = "청바지 추천"
candidates = ["데님 바지", "슬랙스", "남성용 청바지", "바지 스타일 추천", "티셔츠"]

query_embedding = model.encode(query, convert_to_tensor=True)
candidate_embeddings = model.encode(candidates, convert_to_tensor=True)

# 유사도 계산
cos_scores = util.pytorch_cos_sim(query_embedding, candidate_embeddings)[0]

# 상위 유사어 출력
top_k = 3
top_results = torch.topk(cos_scores, k=top_k)

expanded_queries = [candidates[i] for i in top_results.indices]
print("🔍 확장된 질의:", [query] + expanded_queries)

/Users/kimjeongjin/Repo/RAGStudy/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 확장된 질의: ['청바지 추천', '바지 스타일 추천', '남성용 청바지', '데님 바지']


재순위화 (Re-ranking) 예제

transformers의 BERT CrossEncoder를 사용하여 질의-문서쌍 간의 relevance 점수를 계산

In [3]:
from sentence_transformers import CrossEncoder

# CrossEncoder는 문서와 질의 쌍을 입력으로 받아 relevance score 출력
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

query = "청바지 추천"
retrieved_docs = [
    "남성용 데님 팬츠는 다양한 스타일로 제공됩니다.",
    "티셔츠 신상품을 확인하세요.",
    "청바지는 캐주얼한 착장에 적합합니다.",
    "청소기 성능 비교 가이드입니다."
]

# (query, 문서) 쌍 생성
pairs = [(query, doc) for doc in retrieved_docs]

# relevance score 예측
scores = reranker.predict(pairs)

# 점수 기반 정렬
ranked = sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)

print("📊 재순위화 결과:")
for i, (doc, score) in enumerate(ranked):
    print(f"{i+1}. ({score:.4f}) {doc}")

📊 재순위화 결과:
1. (8.4297) 청바지는 캐주얼한 착장에 적합합니다.
2. (8.2407) 티셔츠 신상품을 확인하세요.
3. (8.1952) 남성용 데님 팬츠는 다양한 스타일로 제공됩니다.
4. (8.1321) 청소기 성능 비교 가이드입니다.


고도화된 예제 실행을 위해 추가 라이브러리를 설치하세요.

In [4]:
!pip install sentence-transformers faiss-cpu pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# Query Expansion + FAISS + Re-ranking

from sentence_transformers import SentenceTransformer, CrossEncoder, util
import faiss
import numpy as np

# 문서 컬렉션
documents = [
    "청바지는 다양한 스타일로 출시되고 있습니다.",
    "데님은 오래 입을 수 있는 바지 재질입니다.",
    "스마트폰 신제품이 곧 출시됩니다.",
    "남성용 청바지는 겨울철에도 따뜻합니다.",
    "티셔츠와 바지는 잘 어울리는 조합입니다.",
    "세탁기 추천 상품 모음입니다."
]

# 1. 문서 임베딩 생성
bi_encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
document_embeddings = bi_encoder.encode(documents, convert_to_numpy=True)

# 2. FAISS 인덱스 생성 및 문서 삽입
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# 3. 질의 및 질의 확장
query = "청바지 추천"
expansion_candidates = ["데님 바지", "남성용 청바지", "바지 스타일 추천"]
expanded_queries = [query] + expansion_candidates

# 4. 확장 질의로 검색 수행
top_k = 3
retrieved_indices = set()
for q in expanded_queries:
    q_emb = bi_encoder.encode([q])
    _, I = index.search(q_emb, top_k)
    for i in I[0]:
        retrieved_indices.add(i)

retrieved_docs = [documents[i] for i in retrieved_indices]

# 5. CrossEncoder로 재순위화
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
rerank_pairs = [(query, doc) for doc in retrieved_docs]
scores = cross_encoder.predict(rerank_pairs)

# 6. 결과 출력
ranked = sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)

print("📊 최종 재순위화 결과:")
for i, (doc, score) in enumerate(ranked, 1):
    print(f"{i}. ({score:.4f}) {doc}")

📊 최종 재순위화 결과:
1. (8.2143) 데님은 오래 입을 수 있는 바지 재질입니다.
2. (8.1932) 남성용 청바지는 겨울철에도 따뜻합니다.
3. (8.0115) 세탁기 추천 상품 모음입니다.
4. (7.8539) 청바지는 다양한 스타일로 출시되고 있습니다.
5. (7.5124) 스마트폰 신제품이 곧 출시됩니다.
